In [23]:
using BSON
using CUDA
using DrWatson: struct2dict
using Flux
using Flux: @functor, chunk
using Flux.Losses: logitbinarycrossentropy
using Flux.Data: DataLoader
using Images
using Logging: with_logger
using MLDatasets
using Parameters: @with_kw
using ProgressMeter: Progress, next!
using Random
using JSServe
using WGLMakie
using Statistics

In [10]:
Page(exportable=true, offline=true)

Page(Session(Base.RefValue{Union{Nothing, WebSockets.WebSocket, IOBuffer}}(nothing), Dict{String, Tuple{Bool, Observable}}(), Dict{Symbol, Any}[], Set{JSServe.Asset}(), JSServe.JSCode[], "dbeb90aa-0776-4c97-9980-fc9428779f2c", Channel{Bool}(1), JSServe.init_session, JSServe.UrlSerializer(true, nothing, true, "http://localhost:9284", true), Base.RefValue{Union{Nothing, JSServe.JSException}}(nothing), Observable{Union{Nothing, Dict{String, Any}}} with 0 listeners. Value:
nothing, Observable{Bool} with 0 listeners. Value:
false, Observables.ObserverFunction[], Dict{String, WeakRef}()), Dict{String, Session}(), true, true)

In [11]:
@with_kw mutable struct Args
    η = 1e-3                # learning rate
    λ = 0.01f0              # regularization paramater
    batch_size = 128        # batch size
    sample_size = 10        # sampling size for output    
    epochs = 60             # number of epochs
    seed = 0                # random seed
    cuda = true             # use GPU
    input_dim = 28^2        # image size
    latent_dim = 8          # latent dimension
    hidden_dim = 500        # hidden dimension
    verbose_freq = 10       # logging for every verbose_freq iterations
    save_path = "output"    # results path
end

Args

In [12]:
function get_data(batch_size)
    xtrain, ytrain = MLDatasets.MNIST(:train)[:]
    xtrain = reshape(xtrain, 28^2, :)
    DataLoader((xtrain, ytrain), batchsize=batch_size, shuffle=true)
end

struct Encoder
    linear
    μ
    logσ
end
@functor Encoder
    
Encoder(input_dim::Int, latent_dim::Int, hidden_dim::Int) = Encoder(
    Dense(input_dim, hidden_dim, tanh),   # linear
    Dense(hidden_dim, latent_dim),        # μ
    Dense(hidden_dim, latent_dim),        # logσ
)

function (encoder::Encoder)(x)
    h = encoder.linear(x)
    encoder.μ(h), encoder.logσ(h)
end

Decoder(input_dim::Int, latent_dim::Int, hidden_dim::Int) = Chain(
    Dense(latent_dim, hidden_dim, tanh),
    Dense(hidden_dim, input_dim)
)

function reconstuct(encoder, decoder, x)
    μ, logσ = encoder(x)
    z = μ + gpu(randn(Float32, size(logσ))) .* exp.(logσ)
    μ, logσ, decoder(z)
end

function model_loss(encoder, decoder, λ, x)
    μ, logσ, decoder_z = reconstuct(encoder, decoder, x)
    len = size(x)[end]
    # KL-divergence
    kl_q_p = 0.5f0 * sum(@. (exp(2f0 * logσ) + μ^2 -1f0 - 2f0 * logσ)) / len

    logp_x_z = -logitbinarycrossentropy(decoder_z, x, agg=sum) / len
    # regularization
    reg = λ * sum(x->sum(x.^2), Flux.params(decoder))
    
    -logp_x_z + kl_q_p + reg
end

function convert_to_image(x, y_size)
    Gray.(permutedims(vcat(reshape.(chunk(x |> cpu, y_size), 28, :)...), (2, 1)))
end

convert_to_image (generic function with 1 method)

In [13]:
# load hyperparamters
args = Args(;)
args.seed > 0 && Random.seed!(args.seed)

# load MNIST images
loader = get_data(args.batch_size)

# initialize encoder and decoder
encoder = Encoder(args.input_dim, args.latent_dim, args.hidden_dim) |> gpu
decoder = Decoder(args.input_dim, args.latent_dim, args.hidden_dim) |> gpu

# ADAM optimizer
opt = ADAM(args.η)

# parameters
ps = Flux.params(encoder.linear, encoder.μ, encoder.logσ, decoder)

!ispath(args.save_path) && mkpath(args.save_path)

# fixed input
original, _ = first(get_data(args.sample_size^2))
original = original |> gpu
image = convert_to_image(original, args.sample_size)
image_path = joinpath(args.save_path, "original.png")
save(image_path, image)

# training
train_steps = 0
@info "Start Training, total $(args.epochs) epochs"
for epoch = 1:args.epochs
    @info "Epoch $(epoch)"
    progress = Progress(length(loader))

    for (x, _) in loader 
        loss, back = Flux.pullback(ps) do
            model_loss(encoder, decoder, args.λ, x |> gpu)
        end
        grad = back(1f0)
        Flux.Optimise.update!(opt, ps, grad)
        # progress meter
        next!(progress; showvalues=[(:loss, loss)]) 

        train_steps += 1
    end
    
    # save image
    _, _, rec_original = reconstuct(encoder, decoder, original)
    rec_original = sigmoid.(rec_original)
    image = convert_to_image(rec_original, args.sample_size)
    image_path = joinpath(args.save_path, "epoch_$(epoch).png")
    save(image_path, image)
    @info "Image saved: $(image_path)"
end

# save model
model_path = joinpath(args.save_path, "model.bson") 
let encoder = cpu(encoder), decoder = cpu(decoder), args=struct2dict(args)
    BSON.@save model_path encoder decoder args
    @info "Model saved: $(model_path)"
end

┌ Warning: ProgressMeter by default refresh meters with additional information in IJulia via `IJulia.clear_output`, which clears all outputs in the cell. 
│  - To prevent this behaviour, do `ProgressMeter.ijulia_behavior(:append)`. 
│  - To disable this warning message, do `ProgressMeter.ijulia_behavior(:clear)`.
└ @ ProgressMeter /home/server/.julia/packages/ProgressMeter/sN2xr/src/ProgressMeter.jl:618
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
  loss:  139.23619
┌ Info: Image saved: output/epoch_60.png
└ @ Main In[13]:52
┌ Info: Model saved: output/model.bson
└ @ Main In[13]:59


In [14]:
BSON.@load "output/model.bson" encoder decoder args
args = Args(; args...)
encoder, decoder = encoder |> gpu, decoder |> gpu
# load MNIST images
loader = get_data(args.batch_size)

DataLoader{Tuple{Matrix{Float32}, Vector{Int64}}, Random._GLOBAL_RNG, Val{nothing}}((Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [5, 0, 4, 1, 9, 2, 1, 3, 1, 4  …  9, 2, 9, 5, 1, 8, 3, 5, 6, 8]), 128, false, true, true, false, Val{nothing}(), Random._GLOBAL_RNG())

In [48]:
x = loader.data[1][:,1:1024]
y = loader.data[2][1:1024]
μ, logσ2 = encoder(x |> gpu)
μ = collect(μ)

8×1024 Matrix{Float32}:
  1.71761   -0.672872  -0.187794  …   0.203887   -0.0861286  -1.13624
 -0.80089    1.10315   -2.70845       0.0470791  -1.35881    -1.73645
  1.23401   -0.496134  -3.83533       1.41148     0.0902791  -0.0617696
 -1.57174   -2.72571   -0.83833      -2.11684    -1.14632    -3.02955
  0.711355   0.449077   1.9368        0.69815    -0.421218    0.922298
 -0.727522  -1.84259    0.861007  …  -0.811631    2.61743     2.1328
 -0.105939  -0.520767   0.174523      0.234196   -1.6377      1.82148
 -0.249284   0.985724  -2.71736      -0.472547    0.682166    2.50855

In [49]:
VQ = 2.0 .* std(μ,dims=2) .* ((μ .> mean(μ,dims=2)).-0.5)

8×1024 Matrix{Float64}:
  1.27928  -1.27928  -1.27928   1.27928  …   1.27928   1.27928  -1.27928
 -1.37785   1.37785  -1.37785  -1.37785     -1.37785  -1.37785  -1.37785
  1.48358  -1.48358  -1.48358   1.48358      1.48358  -1.48358  -1.48358
 -1.21641  -1.21641  -1.21641   1.21641     -1.21641  -1.21641  -1.21641
  1.45779  -1.45779   1.45779   1.45779      1.45779  -1.45779   1.45779
 -1.51199  -1.51199   1.51199   1.51199  …  -1.51199   1.51199   1.51199
  1.21189  -1.21189   1.21189   1.21189      1.21189  -1.21189   1.21189
 -1.42336   1.42336  -1.42336  -1.42336     -1.42336   1.42336   1.42336

In [62]:
ŷ = decoder(VQ |> gpu);
ŷ₁ = collect(reshape(ŷ[:,5],28,28))
surface(ŷ₁)

In [60]:
y[3]

4

In [18]:
WGLMakie.activate!()
plt = meshscatter(μ[1,:],μ[7,:],μ[8,:],color=y, colormap=:darktest)